Data 620 
Project 3  
Hantz Angrand  
Name Gender Classifier

In [1]:
import nltk, re, pprint
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.collocations import BigramCollocationFinder
from collections import Counter
import random

1.-  Create the training, dev_test and test set for the names corpus

In [82]:
def last_char(name):
    return {'last_letter': name[-1]}

last_char('Shrek')


{'last_letter': 'k'}

In [83]:
from nltk.corpus import names
nltk.download('names')
#set the seeds

random.seed(1234)
#Divide data

names = ([(name,'male') for name in names.words('male.txt')] +
        [(name, 'female') for name in names.words('female.txt')])

random.shuffle(names)




[nltk_data] Downloading package names to
[nltk_data]     C:\Users\hangr\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!


2.- Feature Sets I

In [84]:
featuresets = [(last_chars(n),gender) for (n,gender) in names]
test_names = featuresets[:500]
devtest_names = featuresets[500 : 1000]
train_names = featuresets[1000:]

In [85]:
test_names[:5]

[({'last_letter': 'r'}, 'male'),
 ({'last_letter': 'o'}, 'male'),
 ({'last_letter': 's'}, 'male'),
 ({'last_letter': 'n'}, 'female'),
 ({'last_letter': 'p'}, 'male')]

In [86]:
devtest_names[:5]

[({'last_letter': 'e'}, 'female'),
 ({'last_letter': 'e'}, 'female'),
 ({'last_letter': 'e'}, 'female'),
 ({'last_letter': 's'}, 'male'),
 ({'last_letter': 'y'}, 'male')]

In [49]:
#Check the length of each divided dataset
print(len(test_names),len(devtest_names),len(train_names))

500 500 6944


In [50]:
#Print first ten names in names dataset
names[:10]

[('Butler', 'male'),
 ('Umberto', 'male'),
 ('Aloysius', 'male'),
 ('Katalin', 'female'),
 ('Kip', 'male'),
 ('Alma', 'female'),
 ('Dennis', 'male'),
 ('Shandie', 'female'),
 ('Rosy', 'female'),
 ('Kaylyn', 'female')]

Naive Bayes classifier for feature set I

In [87]:
##Train our model using the train_set dataset

navbClassifier = nltk.NaiveBayesClassifier.train(train_set)

In [88]:
#Test it out with some names
#from nltk import NaiveBayesClassifier
navbClassifier.classify(last_chars('Neo'))


'female'

In [89]:
#Test the accuracy using the devtest_set

print(nltk.classify.accuracy(navbClassifier, devtest_names))

0.638


In [90]:
#Test set accuracy
print("Test Set accuracy", nltk.classify.accuracy(navbClassifier, test_names))

Test Set accuracy 0.6


In [ ]:
A slight improvement on the test accuracy

In this case which feature that are most effective on distinguishing the name's genders

In [91]:
navbClassifier.show_most_informative_features(5)

Most Informative Features
                 suffix1 = 'k'              male : female =     46.2 : 1.0
                 suffix1 = 'a'            female : male   =     36.6 : 1.0
                 suffix1 = 'f'              male : female =     15.4 : 1.0
                 suffix1 = 'v'              male : female =      9.9 : 1.0
                 suffix1 = 'p'              male : female =      9.9 : 1.0


3.-Improve our accuracy by changing our feature set.  Our new feature set will take the last 2 characters where the last one is a vowel.

In [107]:
def gender_features_2(name):
    features = {}
    features["firstletter"] = name[0].lower()
    features["lastletter"] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count(%s)" % letter] = name.lower().count(letter)
        features["has(%s)" % letter] = (letter in name.lower())
    return features

In [112]:
gender_features_2('Neo')

{'firstletter': 'n',
 'lastletter': 'o',
 'count(a)': 0,
 'has(a)': False,
 'count(b)': 0,
 'has(b)': False,
 'count(c)': 0,
 'has(c)': False,
 'count(d)': 0,
 'has(d)': False,
 'count(e)': 1,
 'has(e)': True,
 'count(f)': 0,
 'has(f)': False,
 'count(g)': 0,
 'has(g)': False,
 'count(h)': 0,
 'has(h)': False,
 'count(i)': 0,
 'has(i)': False,
 'count(j)': 0,
 'has(j)': False,
 'count(k)': 0,
 'has(k)': False,
 'count(l)': 0,
 'has(l)': False,
 'count(m)': 0,
 'has(m)': False,
 'count(n)': 1,
 'has(n)': True,
 'count(o)': 1,
 'has(o)': True,
 'count(p)': 0,
 'has(p)': False,
 'count(q)': 0,
 'has(q)': False,
 'count(r)': 0,
 'has(r)': False,
 'count(s)': 0,
 'has(s)': False,
 'count(t)': 0,
 'has(t)': False,
 'count(u)': 0,
 'has(u)': False,
 'count(v)': 0,
 'has(v)': False,
 'count(w)': 0,
 'has(w)': False,
 'count(x)': 0,
 'has(x)': False,
 'count(y)': 0,
 'has(y)': False,
 'count(z)': 0,
 'has(z)': False}

In [108]:
feature_set =[(gender_features_2(n), g) for (n,g) in names]
train_set = feature_set[500:]
devtest_set= feature_set[500:1000]
test_set = feature_set[:500]

In [113]:
#Train our model
navbClassifier2 = nltk.NaiveBayesClassifier.train(train_set)
print("Devtest accuracy", nltk.classify.accuracy(navbClassifier2, devtest_set))

Devtest accuracy 0.764


In [114]:
#Test Set accuracy
print("Test set accuracy", nltk.classify.accuracy(navbClassifier2, test_set))

Test set accuracy 0.764


Examine individual error cases where the model predicted the wrong label.

In [120]:
errs=[]
for (name,tag) in names:
    #print(name,tag)
    guess = navbClassifier2.classify(gender_features_2(name))
    if guess != tag:
        errs.append((tag, guess, name))

for (tag, guess, name) in sorted(errs):
    print ('correct=%-8s guess=%-8s name=%-30s' %(tag, guess, name))

correct=female   guess=male     name=Addis                         
correct=female   guess=male     name=Ag                            
correct=female   guess=male     name=Agnes                         
correct=female   guess=male     name=Amargo                        
correct=female   guess=male     name=Amber                         
correct=female   guess=male     name=Ambur                         
correct=female   guess=male     name=Aphrodite                     
correct=female   guess=male     name=Ardath                        
correct=female   guess=male     name=Ardis                         
correct=female   guess=male     name=Ardith                        
correct=female   guess=male     name=Ardys                         
correct=female   guess=male     name=Ardyth                        
correct=female   guess=male     name=Astrid                        
correct=female   guess=male     name=Astrix                        
correct=female   guess=male     name=Aubrey     

correct=male     guess=female   name=Maddy                         
correct=male     guess=female   name=Madison                       
correct=male     guess=female   name=Maison                        
correct=male     guess=female   name=Manish                        
correct=male     guess=female   name=Manny                         
correct=male     guess=female   name=Manuel                        
correct=male     guess=female   name=Marcel                        
correct=male     guess=female   name=Marchall                      
correct=male     guess=female   name=Marietta                      
correct=male     guess=female   name=Marilu                        
correct=male     guess=female   name=Marion                        
correct=male     guess=female   name=Marlin                        
correct=male     guess=female   name=Marlon                        
correct=male     guess=female   name=Marmaduke                     
correct=male     guess=female   name=Marshal    

Another function where we combine both methods and look if the consonant english are present

In [122]:
def gender_features_3(name):
    features = {}
    temp_name = name
    eng_cons_clusters = ["bl", "br", "ch", "cl", "cr", "dr", "fl", "fr", "gl", "gr", "pl", "pr", "sc", "sh", "sk", "sl", "sm", "sn", "sp", "st", "sw", "th", "tr", "tw", "wh", "wr", "sch", "scr", "shr", "sph", "spl", "spr", "squ", "str", "thr"]
    features["firstletter"] = name[0].lower() 
    features["lastletter"] = name[-1].lower() 
    features["prefix"] = name[:3].lower() if len(name) > 4 else name[:2].lower() 
    features["suffix"] = name[-3:].lower() if len(name) > 4 else name[-2:].lower()
    clusters = []
    for cluster in eng_cons_clusters[::-1]:
        if cluster in temp_name:
            temp_name = temp_name.replace(cluster, "")
            clusters.append(cluster)
    features["english_consonant_clusters_1"] = clusters[0] if len(clusters) > 0 else None
    features["english_consonant_clusters_2"] = clusters[1] if len(clusters) > 1 else None
    features["english_consonant_clusters_3"] = clusters[2] if len(clusters) > 2 else None
    return features

In [123]:
gender_features_3('Shrek')

{'firstletter': 's',
 'lastletter': 'k',
 'prefix': 'shr',
 'suffix': 'rek',
 'english_consonant_clusters_1': None,
 'english_consonant_clusters_2': None,
 'english_consonant_clusters_3': None}

In [124]:
featureset=[(gender_features_3(n),g) for (n,g) in names]
train_set=featureset[500:]
devtest_set=featureset[500:1000]
test_set=featureset[500:]

In [125]:
#Train the model
navbclassifier3=nltk.NaiveBayesClassifier.train(train_set)
#Devtest Accuracy
print("Devtest accuracy", nltk.classify.accuracy(navbclassifier3, devtest_set))

Devtest accuracy 0.884


In [126]:
#Test set accuracy
print("Test Set Accuracy", nltk.classify.accuracy(navbclassifier3, test_set))

Test Set Accuracy 0.8835303600214938


By combining the first two methods and by looking if the english consonant is present we improve the accuracy of your model.  The last model will predict the gender almost 90% of the time.